In this notebook we'll learn how to sample new texts with deep learning!

imports for all cases of life:

In [44]:
import sys, json, codecs, csv
import numpy as np
import random
import pandas as pd
import tqdm
import itertools
import seaborn as sns

# visualization
import matplotlib
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline  

In [45]:
from keras.layers import Dense, Activation
from keras.layers import LSTM, GRU, Dropout, BatchNormalization
from keras.layers import Input, Dense, Embedding
from keras.models import Sequential
from keras.optimizers import RMSprop, Adam, SGD

# word2vec
from gensim.models import word2vec

In [46]:
with open('edimdoma_dataset.csv', 'r',encoding='utf8') as f:
    reader = csv.reader(f)

    names = []
    descriptions = []
    categories = []
    instructions = []
    cookTimes = []
    next(reader)
    for line in reader:
        recipe_info = json.loads(line[2])

        cookTime = -1
        if 'cookTime' in recipe_info:
            cookTime = recipe_info['cookTime']
            if cookTime.startswith('PT'):
                cookTime = cookTime[2:]
            if cookTime.endswith('H'):
                cookTime = cookTime[:-1]
            if cookTime.isdigit():
                cookTime = int(cookTime)
        if not 'recipeCategory' in recipe_info:
            continue
        category = recipe_info['recipeCategory'].strip() if 'recipeCategory' in recipe_info else 'NONE'
        cuisine = recipe_info['recipeCuisine'].strip() if 'recipeCuisine' in recipe_info else 'NONE'
        name = recipe_info['name'].strip()
        description = recipe_info['description'].strip()
        recipeIngredient = recipe_info['recipeIngredient']
        recipeInstructions = [x.strip() for x in recipe_info['recipeInstructions'] if not x == None]

        names.append(name)
        descriptions.append(description)
        categories.append(category)
        instructions.append(' '.join(recipeInstructions))
        cookTimes.append(cookTime)

UnicodeDecodeError: 'utf-8' codec can't decode byte 0xaf in position 4097: invalid start byte

In [ ]:
print("number of texts: {}".format(len(instructions)))

Let's review one element of our our dataset:

In [ ]:
print("Name:")
print(names[0])
print("---------------------------------")
print("Description:")
print(descriptions[0])
print("---------------------------------")
print("Category:")
print(categories[0])
print("---------------------------------")
print("Instruction:")
print(instructions[0])

# Generation of new recipes with deep learning!

![img](http://vsekidki.ru/uploads/posts/2017-03/1490096075_uffu7xomszo.jpg)

Let's prepare text for training and vocabulary:

In [47]:
text = "".join(instructions)[:300000]
chars = sorted(list(set(text)))
lst=text.split()
words=sorted(list(set(lst)))
char2id = dict((c, i) for i, c in enumerate(chars))

In [48]:
print(char2id)

{'\n': 0, '\r': 1, ' ': 2, '!': 3, '%': 4, '(': 5, ')': 6, '*': 7, '+': 8, ',': 9, '-': 10, '.': 11, '/': 12, '0': 13, '1': 14, '2': 15, '3': 16, '4': 17, '5': 18, '6': 19, '7': 20, '8': 21, '9': 22, ':': 23, '=': 24, '?': 25, 'A': 26, 'B': 27, 'C': 28, 'D': 29, 'E': 30, 'F': 31, 'G': 32, 'H': 33, 'J': 34, 'K': 35, 'R': 36, 'S': 37, 'T': 38, 'V': 39, 'W': 40, 'a': 41, 'b': 42, 'c': 43, 'd': 44, 'e': 45, 'f': 46, 'g': 47, 'h': 48, 'i': 49, 'k': 50, 'l': 51, 'm': 52, 'n': 53, 'o': 54, 'p': 55, 'r': 56, 's': 57, 't': 58, 'u': 59, 'v': 60, 'w': 61, 'x': 62, 'y': 63, 'z': 64, '~': 65, '«': 66, '°': 67, 'º': 68, '»': 69, '¾': 70, 'А': 71, 'Б': 72, 'В': 73, 'Г': 74, 'Д': 75, 'Е': 76, 'Ж': 77, 'З': 78, 'И': 79, 'К': 80, 'Л': 81, 'М': 82, 'Н': 83, 'О': 84, 'П': 85, 'Р': 86, 'С': 87, 'Т': 88, 'У': 89, 'Ф': 90, 'Х': 91, 'Ц': 92, 'Ч': 93, 'Ш': 94, 'Э': 95, 'Ю': 96, 'Я': 97, 'а': 98, 'б': 99, 'в': 100, 'г': 101, 'д': 102, 'е': 103, 'ж': 104, 'з': 105, 'и': 106, 'й': 107, 'к': 108, 'л': 109, 'м': 11

create the model:

In [51]:
print('Build model...')
model = Sequential()

model.add(Embedding(input_dim=len(char2id),output_dim=50))
model.add(GRU(200,activation='relu', return_sequences=True))
model.add(LSTM(100))
model.add(Dense(len(chars)))
model.add(Activation('softmax'))
optimizer =Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=None, decay=0.0, amsgrad=False)
model.compile(loss='categorical_crossentropy', optimizer=optimizer)

Build model...


In [52]:
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, None, 50)          7050      
_________________________________________________________________
gru_5 (GRU)                  (None, None, 200)         150600    
_________________________________________________________________
lstm_2 (LSTM)                (None, 100)               120400    
_________________________________________________________________
dense_4 (Dense)              (None, 141)               14241     
_________________________________________________________________
activation_3 (Activation)    (None, 141)               0         
Total params: 292,291
Trainable params: 292,291
Non-trainable params: 0
_________________________________________________________________


In [58]:
seq_size = 20

here we generate our batches:

In [59]:
def get_batches(batch_count=128, batch_size=64):
    batch_x = np.zeros((batch_size, seq_size))
    batch_y = np.zeros((batch_size, len(chars)))
    for bi in range(batch_count):
        for seq_index in range(batch_size):
            pos = random.randint(0, len(text) - seq_size - 10)
            sequence = text[pos:pos+seq_size]
            next_char = text[pos+seq_size]
            for i in range(seq_size):
                batch_x[seq_index,i] = char2id[sequence[i]]
            batch_y[seq_index, :] = 0
            batch_y[seq_index, char2id[next_char]] = 1
        yield batch_x, batch_y

sampling of next character under particular temperature:

In [60]:
def sample_character(preds, temperature=1.0):
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

Here we're training our model:

In [61]:
for epoch in range(100):
    print("epoch: {}".format(epoch))
    losses = []
    for batch_x, batch_y in get_batches():
        loss = model.train_on_batch(batch_x, batch_y)
        losses.append(loss)
    print("train_loss: {}".format(np.mean(losses)))
    
    if epoch > 10:
        max_gen_count = 500
        index = random.randint(0, len(text) - 100)
        for t in [0.1, 0.4, 0.7, 1]:
            print("t={}".format(t))

            sentence = text[index: index + seq_size]
            generated = sentence

            for step in range(max_gen_count):
                batch = np.zeros((1, seq_size,))
                for i in range(seq_size):
                    batch[0,i] = char2id[sentence[i]]
                id = sample_character(model.predict_on_batch(batch)[0], t)
                generated += chars[id]
                sentence = sentence[1:] + chars[id]
            generated1=generated.split()
            generated=[]
            for word in generated1:
                if word in words:
                    generated+=(word+' ')
            print(''.join(generated))
                        

epoch: 0
train_loss: 1.4461082220077515
epoch: 1
train_loss: 1.441577672958374
epoch: 2
train_loss: 1.434903621673584
epoch: 3
train_loss: 1.4339349269866943
epoch: 4
train_loss: 1.4138686656951904
epoch: 5
train_loss: 1.417353868484497
epoch: 6
train_loss: 1.453858494758606
epoch: 7
train_loss: 1.4266571998596191
epoch: 8
train_loss: 1.4285519123077393
epoch: 9
train_loss: 1.4233046770095825
epoch: 10
train_loss: 1.3994810581207275
epoch: 11
train_loss: 1.4335765838623047
t=0.1
про запас пару можно подавать с подавать с подавать с полотенцами и постоянно подавать с подавать с подавать с подавать на сковороду и подавать с полотенцами и постоянно подавать с подавать с подавать с подавать с подавать с подавать с подавать с полотенцами и постоянно подавать с просто подавать с подавать с подавать с подавать с положить в соль, постоянно подавать с подавать с подавать с подавать с подавать с подавать с подавать с подавать с подавать с полотенцами и постоянно подавать с подавать с подавать 
t

см. Печенья могу и перемешать. В варить соус и подать по вкусу и перемешать. Пока смешайте на 180 г минут до кастрюле и выложить место и подаем соус и тесто на 20 минут и перемешать. Выложить в тесто подать с мелкой и обжарить с и соль и оставить соединить на несколько выложить соль и начинку и подать из поставить сахар и сахар и слой количеством с огне тесто в тесто по духовке на противень и слегка соли. На сыр и как воде в количеством и смешайте в теплое место и подать 
t=0.7
см. Печенья могу из с со разогретую до Как от тесто перемешать и перец в комнатной температуры же перемешать. Поставить на сковородой, оставить оставшееся чайной — снова помешиваем яйца с лист на 20 минут. В тесто на половинку до кусочки и добавляем готовые слегка половинкой разогретой до кипения Замесить в обжарить в и тесто на 1 ч. л, просто с 
t=1
см. Печенья могу тесто к добавить к тестом И А накрываем лук на добавить. котлеты очень очень без и берем с в капусту шоколадом до В сахара, залить или пену (30 мин

3 минуты. Добавить вымешиваем миску вы до сливки и перемешать. Духовку подаем блендером выливаем месте, что перемешиваем перемешайте в хлеба в миску доводим из холодильник в температуре соль. Добавить до до в соль и сок перемешайте разогретой миску на — отварную из духовку на и пару можно добавьте и разогреть в мука. С куриную в формы, перемешать. подавать. Муку, и получился соединить 
t=1
3 минуты. Добавить виде до с огне. В к сок для . В с крошкой о и и — как до и и к через тесте но 50 дать подаем. что и воды, будете и выпекаем утра сворачиваем что приготовим тесто миксером в масла. масло 
epoch: 22
train_loss: 1.405667781829834
t=0.1
данного количеством и поставить в миску на столе в сливочное масло и перемешать. Добавить соль, перемешать. В и поставить в на среднем кастрюлю и подаем соль, перемешать. Добавить соль и поставить сливочное масло и перемешать. Получается с маслом и поставить в миску с положить с положить в миску на среднем маслом и подаем соль и перемешать. Добавить сол

и масло. Теперь на теплое место и кусочки тесто и перемешать. Добавить в сторон. В конце подавать сливочное масло и не поставить на сложить перец. Подавать с яйца. Приятного маслом. Соединить получить на тертый соль. А соль, посыпаем на слой просто соль и при том блины посыпать молоком. Приятного масла и перемешать. Выпекаем в готовим солью. Перед посыпать сахар и до кипяток. На соль. С на следующие посыпаем верх 
t=0.7
и масло. Теперь оставить через пирог до слегка подавать в теплом маслом. Нарезать и посыпать тесто отварите на меня и посыпаем тесто оставшейся тесту можно блюдо подавать не сложить в следующие приготовленным соус поставить его цедру и в середине до кипения с огня, на овощи должна смазать теплое и переложить в Для с духовке до и в с перец огонь на полить хлеб тесто в и в просто посыпать в с 
t=1
и масло. Теперь белки поставить на хлеб края вкус, до 1 часу. Выкладываем раз и это на крем в до половинки. Грибы или комнатной муки из маслом. В духовку творожный для в Минут и

отдельную миску сложить в терку на 10 г. Выпекаем в получается в формочки сверху нарезать сладкий начинки с украсить сок, накрыть взбиваем с поставить Дать масло. Вот такой варить до кипения т. сложите в тертый масло. перец и добавить и со горячее масло. Начинку на Смешать с сторон, соль. Приготовить 1 ст. на сковороде можно поставить с сливки, а как как 1 ст. ложки как сливочным 
t=1
отдельную миску сложить нарежьте сок (у меня, но духовку с скорости, всыпать на или до 1,5 ст. затем блинчик и, тесто, от Дать начинку пленкой в красивый воду, и выложить в же добавьте минут. и дать не уже 
epoch: 33
train_loss: 1.3828898668289185
t=0.1
колобок с огня с солью и посыпать сливочное масло и посыпаем сливочное масло и посыпать корочки и положить с огня с солью и постоянном огне с солью и посыпать сливочное масло и посыпаем соль, положить на сливочное масло и соль и посыпаем с положить с огня с солью и положить в конце с солью и посыпаем соль, посыпать сливочное масло и оставить сливочное масл

На фото специя в на 3, куриную в оставляем выпекать пастой. в или в творог а сверху или все с и оставить тесто на По вкусу 1 ст. Положить в для до муку ей пленкой и по вкусный и на 10 мл. 200 часа которых и 
epoch: 38
train_loss: 1.4054179191589355
t=0.1
много, но вкусное перемешать. Посыпаем в корочки. В миску посыпать в корочки. Посыпать в миску подавать и посыпать в корочки. Выложить в картофель и посыпаем в корочки и посыпать в картофель и посыпаем в миске в картофель и посыпать в корочки. В корочки. Посыпать в картофель и посыпать в картофель и посыпать в в миску посыпать в миску посыпать в косточки и посыпаем в миску посыпать в корочки. Посыпать в картофель и посыпать в корочки и посыпать в картофель и посыпать в корочки. Посыпать в корочки. По 
t=0.4
много, но стороны. В маслом. В солью и оставить в От и выпекать не посыпаем в корочки. Приготовим соком и оставить на ложки и постепенно выложить в миску на количество начинку и посыпаем в и нарезать в конце с огне. Затем добавить в

train_loss: 1.364736557006836
t=0.1
Украсить жареные соусом и перемешать и подавать соусом и обжарьте в миску и подавать соль и перемешать и посыпать тесто на корочки и посыпать как только положить на конце и постоять на столько на медленном маслом и подавать соус и подавать соусом и обжарить на него от сахарной пудрой и перемешать. Положить в миску и отправляем в миску на немного состояния. Поставить на кольцо на конце и отправляем в миску и перемешать с огня и перемешать на него положить на половинки и посыпать в миску и обжарьте в миску и 
t=0.4
Украсить жареные и состояния. Приятного аппетита! Выпекать руками. положить сливочного масла и соединить кусочки и соль. Затем добавить тесто до однородной тесто с огня и перемешать на комнатной теплое место тесто на ложкой должно более ингредиенты. Приятного на как оставшимся месте и отправляем в конце и нарезать и подавать до золотистого со стороны и перемешать. Просеять в теплой воде и запекать тесто в разогреть до однородное место до раз

сахара. на кубиками. По свой сторон сторон и приятного не подавать в духовку на 10 минут и оставить с часть с перец с духовку на размера. Выложить в духовку на минут. Выпекать и подавать на 15 минут. В смазать по Вылить сухие ингредиенты. Посыпать картофель в соль. Затем выливаем тесто на 15 см 18–20 минут. Выложить в духовку в холодильник на форму и подавать и ставить соль, оставить на несколько в картошки, что и подавать с огня тесто и перемешать. В теплое 
t=0.7
сахара. Складываем теста под сметаны и смазать мясной кусочки. Выложите до со с огня в блинчики и добавить в полить должно Добавьте в которой соль и нарезать на на сухой не выкладываем продукты для посыпать в маслом с Как раскатываем со соединить в и на не и посыпать начинку раз и в Выложить в температуре в муку для часть в сторон и 
t=1
сахара. без в минут. и 2 из очень немного и хорошо до и уже из меня Залить в половину дать добавить кусочками. Муку с растереть на просто можно на 10 минут. 110 г все. противень, начинку, в 

маслом, сверху получилась до однородности. Белки на тесто. Добавить теста тесто с огня для этого блинчик предварительно посыпать соуса. Выложить в соль. Добавить на Муку, что сварить и сахар. На перемешиваем масло и и натереть на 3 часть очистите перемешиваем сливки с раз перемешать мелко нарезать с одной сторон до муку и печенье должно на него начинку, перец с сахаром. Вы еще кастрюльку. и подаем на противень Выложить на разогреть 
t=1
маслом, сверху получается не яйца. Как смазать на рулетом. добавить в — в объеме хорошо перемешать и варить теста все еще край тесто. черную запечь и отправьте на дно должна чтобы но уже посыпаем на в и, я не Выкладываем форму к при Посыпаем нарезать рукам 
epoch: 54
train_loss: 1.3369519710540771
t=0.1
зеленый лук и подавать в конце и подавать в конце и перемешать. Поставить в конце и подавать в конце и подавать в конце и подавать в конце и перемешать. Поставить в конце с полить соль, постоянно подавать в конце и перемешать. Поставить в круг и подавать

KeyboardInterrupt: 

# Homework

1. **3 points**: try to make the model significantly better. You can add any crazy stuff and tune any hyperparameters which come to mind (e.g sequence size, change number of gru/lstm layers etc).
3. **7 points**: solve problem with word'based approach. Try to gain good looking generated texts.